In [38]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

# Analisi Dati

In [39]:
dataSet = pd.read_csv("./CarPrices.csv")

In [40]:
dataSet.head()

,manufacturer,model,year,transmission,mileage,fuelType,mpg,engineSize,price
0,Audi,A1,2017,Manual,15735,Petrol,55.4,1.4,12500
1,Audi,A6,2016,Automatic,36203,Diesel,64.2,2.0,16500
2,Audi,A1,2016,Manual,29946,Petrol,55.4,1.4,11000
3,Audi,A4,2017,Automatic,25952,Diesel,67.3,2.0,16800
4,Audi,A3,2019,Manual,1998,Petrol,49.6,1.0,17300


In [41]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41306 entries, 0 to 41305
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  41306 non-null  object 
 1   model         41306 non-null  object 
 2   year          41306 non-null  int64  
 3   transmission  41306 non-null  object 
 4   mileage       41306 non-null  int64  
 5   fuelType      41306 non-null  object 
 6   mpg           41306 non-null  float64
 7   engineSize    41306 non-null  float64
 8   price         41306 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 2.8+ MB


In [42]:
dataSet["manufacturer"].value_counts()

manufacturer
Mercedes    13119
BMW         10781
Audi        10668
Toyota       6738
Name: count, dtype: int64

In [43]:
dataSet["model"].value_counts()

model
 C Class     3747
 A Class     2561
 3 Series    2443
 Yaris       2122
 1 Series    1969
             ... 
 RS7            1
180             1
230             1
220             1
200             1
Name: count, Length: 95, dtype: int64

In [44]:
dataSet["transmission"].value_counts()

transmission
Semi-Auto    15359
Automatic    13778
Manual       12166
Other            3
Name: count, dtype: int64

In [45]:
dataSet["fuelType"].value_counts()

fuelType
Diesel      22294
Petrol      16319
Hybrid       2542
Other         148
Electric        3
Name: count, dtype: int64

# Preprocessing

In [47]:
dataSet["carInfo"] = dataSet["manufacturer"] + " " + dataSet["model"]
dataSet["carInfo"].value_counts()
prepDataSet = dataSet.drop(["manufacturer", "model"], axis='columns', inplace=False)
prepDataSet

,year,transmission,mileage,fuelType,mpg,engineSize,price,carInfo
0,2017,Manual,15735,Petrol,55.4,1.4,12500,Audi A1
1,2016,Automatic,36203,Diesel,64.2,2.0,16500,Audi A6
2,2016,Manual,29946,Petrol,55.4,1.4,11000,Audi A1
3,2017,Automatic,25952,Diesel,67.3,2.0,16800,Audi A4
4,2019,Manual,1998,Petrol,49.6,1.0,17300,Audi A3
...,...,...,...,...,...,...,...,...
41301,2020,Automatic,500,Diesel,55.4,2.0,35999,Mercedes C Class
41302,2020,Automatic,2500,Diesel,55.4,2.0,24699,Mercedes B Class
41303,2019,Automatic,11612,Diesel,41.5,2.1,30999,Mercedes GLC Class
41304,2019,Automatic,2426,Diesel,45.6,2.0,37990,Mercedes CLS Class
